In [28]:
# Imports
from hmm import HMM1, State

### Initial implementation of Hidden Markov Model

In [29]:
# Fixed transitions for example
t_pb = {}
t_pb['T'] = {}
t_pb['F'] = {}
# to state X from state Y
t_pb['T']['T'] = 0.7
t_pb['T']['F'] = 0.3
t_pb['F']['T'] = 0.3 # 1 - self.t_pb['T']['T']
t_pb['F']['F'] = 0.7 # 1 - self.t_pb['T']['F']

In [30]:
h = HMM1(t_pb)
print(h.t_pb)

state = {"T": 1, "F": 0}
print(state)
# find converging distribution here
for _ in range(20):
    state = h.transition(state)
    print(state)


{'T': {'T': 0.7, 'F': 0.3}, 'F': {'T': 0.3, 'F': 0.7}}
{'T': 1, 'F': 0}
{'T': 0.7, 'F': 0.3}
{'T': 0.58, 'F': 0.42}
{'T': 0.532, 'F': 0.46799999999999997}
{'T': 0.5128, 'F': 0.48719999999999997}
{'T': 0.50512, 'F': 0.49488}
{'T': 0.502048, 'F': 0.49795199999999995}
{'T': 0.5008192, 'F': 0.4991808}
{'T': 0.50032768, 'F': 0.49967231999999995}
{'T': 0.5001310720000001, 'F': 0.49986892799999993}
{'T': 0.5000524288, 'F': 0.4999475712}
{'T': 0.50002097152, 'F': 0.49997902848}
{'T': 0.500008388608, 'F': 0.499991611392}
{'T': 0.5000033554432, 'F': 0.4999966445568}
{'T': 0.50000134217728, 'F': 0.4999986578227199}
{'T': 0.500000536870912, 'F': 0.4999994631290879}
{'T': 0.5000002147483646, 'F': 0.49999978525163513}
{'T': 0.5000000858993457, 'F': 0.4999999141006539}
{'T': 0.5000000343597382, 'F': 0.4999999656402615}
{'T': 0.5000000137438951, 'F': 0.49999998625610453}
{'T': 0.500000005497558, 'F': 0.4999999945024417}


### New Approach with State class

In [31]:
# Generate states
true = State("T")
false = State("F")

# Link states -> to self, from state dict
true.add({true: 0.7, false: 0.3})
false.add({true: 0.3, false: 0.7})


# true.tfrom(true)
# true.tfrom(false)
print(true)
print(false)

T : [('T', 0.7), ('F', 0.3)]
F : [('T', 0.3), ('F', 0.7)]


In [32]:
def rough_equal(a: float, b: float, dec: int) -> bool:
    """Find if two numbers are equal within a decimal point range `dec`."""
    return round(a, dec) == round(b, dec)

# Ensure linking is correct -- account for python inaccuracies with floats
rough_equal(true.given(false), 1-false.given(false), 3)


True

In [33]:
# h2 = HMM2({true, false})

def state_print(state: dict[State: float]):
    print([(x.id, p) for x,p in state.items()])

state = {true: 1, false: 0}
state_print(state)

# find converging distribution here
for _ in range(20):
    new = {}
    for s in state.keys():
        new[s] = s.total(state)
    state = new
    state_print(state)


[('T', 1), ('F', 0)]
[('T', 0.7), ('F', 0.3)]
[('T', 0.58), ('F', 0.42)]
[('T', 0.532), ('F', 0.46799999999999997)]
[('T', 0.5128), ('F', 0.48719999999999997)]
[('T', 0.50512), ('F', 0.49488)]
[('T', 0.502048), ('F', 0.49795199999999995)]
[('T', 0.5008192), ('F', 0.4991808)]
[('T', 0.50032768), ('F', 0.49967231999999995)]
[('T', 0.5001310720000001), ('F', 0.49986892799999993)]
[('T', 0.5000524288), ('F', 0.4999475712)]
[('T', 0.50002097152), ('F', 0.49997902848)]
[('T', 0.500008388608), ('F', 0.499991611392)]
[('T', 0.5000033554432), ('F', 0.4999966445568)]
[('T', 0.50000134217728), ('F', 0.4999986578227199)]
[('T', 0.500000536870912), ('F', 0.4999994631290879)]
[('T', 0.5000002147483646), ('F', 0.49999978525163513)]
[('T', 0.5000000858993457), ('F', 0.4999999141006539)]
[('T', 0.5000000343597382), ('F', 0.4999999656402615)]
[('T', 0.5000000137438951), ('F', 0.49999998625610453)]
[('T', 0.500000005497558), ('F', 0.4999999945024417)]
